In [3]:
import pandas as pd
import re

In [2]:
easynic = pd.read_csv("./outputs/easy-nic-in/easy-nic.csv")
supremo = pd.read_csv("./outputs/supremo-nic-in/supremo-nic.csv")

In [3]:
easynic["Dataset name"] = ["DOPT(easynic)"] * len(easynic.index)
supremo["Dataset name"] = ["Supremo"] * len(supremo.index)

In [4]:
indexes = []
for i, value in enumerate(supremo["Identity No."].values):
    if str(value)[:2] == "02":
        indexes.append(i)

In [5]:
supremo.iloc[indexes[237]]

Name                                                                      Shri R N Joshi
Service                                                   Indian Administrative Service 
Cadre                                                                           Gujarat 
Allotment Year                                                                      1996
Date of Birth                                                                 08/11/1950
Date of Joining                                                               01/03/2002
Identity No.                                                                  02GJ029016
Source of Recruitment                                                       By Selection
Gender                                                                              Male
Place of Domicile                                                                Gujarat
Mother Tongue                                                                   Gujarati
Languages Known      

In [6]:
def find_in(my_list, value):
    for list_value in my_list:
        if str(value) in str(list_value):
            return True
    
    return False

In [7]:
mapper = {
    "UT": "A G M U T",
    "AP": "Andhra Pradesh",
    "AM": "Assam Meghalya",
    "BH": "Bihar",
    "CG": "Chhasttisgarh",
    "GJ": "Gujarat",
    "HY": "Haryana",
    "HP": "Himachal Pradesh",
    "JK": "Jammu & Kashmir",
    "JH": "Jharkhand",
    "KN": "Karnataka",
    "KL": "Kerala",
    "MP": "Madhya Pradesh",
    "MH": "Maharastra",
    "MN": "Manipur",
    "NL": "Nagaland",
    "OR": "Odisha",
    "PB": "Punjab",
    "RJ": "Rajasthan",
    "SK": "Sikkim",
    "TN": "Tamil Nadu",
    "TG": "Telangana",
    "TR": "Tripura",
    "UP": "Uttar Pradesh",
    "UD": "Uttarakhand",
    "WB": "West Bengal"
}

In [8]:
easynic = easynic.drop(["Unnamed: 0"], axis=1)

In [9]:
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key

In [10]:
for i in easynic.index:
    prefix = "0" if len(str(easynic.loc[i,"Identity No."])) < 6 else ""
    easynic.loc[i, "Identity No."] = str(easynic.loc[i, "Cadre"]) + prefix + str(easynic.loc[i,"Identity No."])

In [11]:
easynic["Identity No."]

0       UT016100
1       UT016500
2       UT016400
3       UT016800
4       UT016900
          ...   
5200    WB118X08
5201    WB118X09
5202    WB118X10
5203    WB118X11
5204    WB118X12
Name: Identity No., Length: 5205, dtype: object

In [12]:
new_cadra = []
for item in easynic["Cadre"]:
    new_cadra.append(mapper[item])

In [13]:
easynic["Cadre"] = new_cadra

In [14]:
qualifications = easynic["Qualification(Subject)"]

In [2]:
def create_compatible_q(my_string):
    q = ""
    s = ""
    item = my_string
    splits = item.split(" ")
    subjects = [""] * len(splits)
    item = item.replace(")(", " ")
    subject_list = re.findall("\((.*?)\)", item)
    counter = 0
    for split in splits:
        if split[len(split)-1] == ")":
            s += subject_list[counter]
            item = item.replace(f"({subject_list[counter]})", "")
            counter += 1
        else:
            s += ""
        s += "|"
    q = item.replace(" ", "|")
    s = s[:len(s)-1]
    return q, s

In [5]:
create_compatible_q("M.A.(Sociology) B.A.(Psychology)")

('M.A.|B.A.', 'Sociology|Psychology')

In [17]:
degree = []
subject = []
division = []
for qualification in qualifications:
    d, s = create_compatible_q(qualification)
    degree.append(d)
    subject.append(s)
    division.append("Not Found")

In [18]:
degree[0]

'M.A.|B.A.'

In [19]:
easynic["Qualification(Subject)"] = degree
easynic["Subject"] = subject
easynic["Division"] = division

In [20]:
exp = easynic["Present Post"]

In [21]:
exp_name = []
location = []
for item in exp:
    item = str(item)
    split = item.split(", ")
    place = split[len(split)-1]
    item = item.replace(", " + place, "")
    exp_name.append(item)
    location.append(place)

In [22]:
location[1]

'M/o Shipping '

In [23]:
easynic["Designation/Level"] = exp_name
easynic["Ministry/Department/Office/Location"] = location
easynic = easynic.rename(columns={
    "Qualification(Subject)" : "Qualification/University/Institute",
    "Post Effect From": "Period(From/To)",
    'Domicile': "Place of Domicile",
    "Dt. of Appointment" : "Date of Joining",
})

In [24]:
easynic

,Name,Identity No.,Date of Joining,Source of Recruitment,IntraIAS User-Id,Qualification/University/Institute,Date of Birth,Allotment Year,Cadre,Place of Domicile,Present Post,Period(From/To),Payscale Remarks,Dataset name,Subject,Division,Designation/Level,Ministry/Department/Office/Location
0,Ms. Rina Ray,UT016100,21-08-1984,RR,rayr,M.A.|B.A.,06-03-1961,1984,A G M U T,Bihar,"Financial Commissioner, GNCT of Delhi",18-11-2019,Level 17 in the Pay Matrix,DOPT(easynic),Sociology||Hons Political Sc.,Not Found,Financial Commissioner,GNCT of Delhi
1,Shri Jalaj Shrivastava,UT016500,21-08-1984,RR,sjalaj,M.Sc.|B.Sc.,08-02-1961,1984,A G M U T,Uttar Pradesh,"Chairman, Inland Waterways Auth of India (IWAI...",17-11-2018,Level 15 in the Pay Matrix,DOPT(easynic),Physics|Physics,Not Found,"Chairman, Inland Waterways Auth of India (IWAI)",M/o Shipping
2,Shri Arvind Ray,UT016400,21-08-1984,RR,raya,M.A.,08-01-1960,1984,A G M U T,Bihar,"Chairman & M D, Delhi State Civil Supplies Cor...",28-02-2019,Level 17 in the Pay Matrix,DOPT(easynic),|Political Sc.,Not Found,"Chairman & M D, Delhi State Civil Supplies Cor...",N C T of Delhi
3,Ms. Shakuntala D Gamlin,UT016800,28-08-1984,RR,gamlins,M.A.|B.A.,29-03-1961,1984,A G M U T,Assam,"Secretary Govt. of India, M/o Social Justice &...",11-10-2017,Level 17 in the Pay Matrix On CD since 11/10/2017,DOPT(easynic),Sociology|History,Not Found,Secretary Govt. of India,M/o Social Justice & Empowerment
4,Shri Parimal Rai,UT016900,26-08-1985,RR,parimalrai,PG|Diploma|M.Tech,05-01-1962,1985,A G M U T,Delhi,"Chief Secretary, Govt, of Goa, Goa",01-01-2019,Level 17 in the Pay Matrix,DOPT(easynic),|Ecology||System Mgt,Not Found,"Chief Secretary, Govt, of Goa",Goa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200,Ms. Tamil Oviya S,WB118X08,01-09-2019,RR,Not found,Graduate,26-09-1993,2019,West Bengal,Tamil Nadu,N.A.,01-09-2018,Level 10 in the Pay Matrix,DOPT(easynic),,Not Found,N.A.,N.A.
5201,Ms. Renu Sogan,WB118X09,01-09-2019,RR,Not found,Graduate,21-03-1994,2019,West Bengal,Rajasthan,N.A.,01-09-2018,Level 10 in the Pay Matrix,DOPT(easynic),,Not Found,N.A.,N.A.
5202,Shri Anuj Pratap Singh,WB118X10,01-09-2019,RR,Not found,Graduate,30-03-1991,2019,West Bengal,Uttar Pradesh,N.A.,01-09-2018,Level 10 in the Pay Matrix,DOPT(easynic),,Not Found,N.A.,N.A.
5203,Ms. Priyanka Singh,WB118X11,01-09-2019,RR,Not found,Graduate,25-03-1992,2019,West Bengal,Uttar Pradesh,N.A.,01-09-2018,Level 10 in the Pay Matrix,DOPT(easynic),,Not Found,N.A.,N.A.


In [25]:
for column in supremo.columns:
    if not column in easynic.columns:
        easynic[column] = ["Not Found"] * len(easynic.index)

In [26]:
for column in supremo.columns:
    if not column in easynic.columns:
        print(column)

In [27]:
temp = supremo

In [28]:
for i in easynic.index:
    row = dict(easynic.iloc[i])
    to_add = {}
    for column in temp.columns:
        to_add[column] = easynic[column][i]
    temp = temp.append(to_add, ignore_index=True)

In [29]:
temp.to_csv("./outputs/combined/test.csv", index=True)